In [ ]:
# Setup the Jupyter version of Dash
from jupyter_plotly_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD Python module (replace 'name' with the actual name)
from AnimalCrud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Set up your MongoDB username and password
username = "aacuser"
password = "SNHU1234"

# Create an instance of your AnimalShelter class
shelter = AnimalShelter(username, password)

# Retrieve all data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)  # Drop the '_id' column

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Ensure there is data and a selected row
    if viewData is not None and index is not None:
        # Create a dataframe from the virtual data
        dff = pd.DataFrame.from_dict(viewData)

        # Extract the selected row
        row = index[0] if index else 0

        # Set up the geolocation chart using Leaflet
        return [
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[30.75, -97.48],  # Austin TX coordinates
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                        children=[
                            dl.Tooltip(dff.iloc[row, 4]),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(dff.iloc[row, 9])
                            ])
                        ]
                    ),
                ],
            )
        ]

# Run the app
app.run_server(debug=True)
